In [ ]:
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions, ResNet50
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pylab as plt
import numpy as np
import tensorflow_datasets as tfds

# Load Datasets

In [6]:
builder = tfds.ImageFolder('/Users/szymon.rucinski/Desktop/AiRoll/data/')

2022-12-22 20:17:11.970385: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


In [7]:
ds = builder.as_dataset(split='train', shuffle_files=True)
# tfds.show_examples(ds, builder.info)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-22 20:17:12.557025: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-22 20:17:12.557207: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Display dataset

In [8]:
datagen = ImageDataGenerator(rescale=1. / 255, horizontal_flip=True,validation_split=0.2)

train_generator = datagen.flow_from_directory(
    directory=r"../data/train/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42,
    subset="training"
)

validation_generator = datagen.flow_from_directory(
    directory=r"../data/train/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42,
    subset="validation"
)

FileNotFoundError: [Errno 2] No such file or directory: '../data/train/'

In [ ]:
from itertools import count
a = train_generator.classes.flatten().tolist()
dist_dict = {}
for label,number in train_generator.class_indices.items():
    dist_dict[label] = a.count(number)
dist_dict

: 

In [ ]:
train_generator.class_indices.items()

: 

In [ ]:
for _ in range(5):
    img, label = train_generator.next()
    print(img.shape)   #  (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

: 

In [ ]:
train_generator.class_indices

: 

# Model Compilation

In [ ]:
resnet50 = ResNet50(include_top=False)
resnet50.trainable = False

: 

In [ ]:
# To ensure that TensorFlow does not retrain the mobile_net_layers, use this code:


: 

In [ ]:
# # Build the neural network and add custom layers.
# neural_net = tf.keras.Sequential([
#   resnet50,
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(7,activation='softmax')
# ])

: 

In [ ]:
resnet_model = tf.keras.Sequential()
pretrained_model = tf.keras.applications.resnet50.ResNet50(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    pooling="avg",
    classes=7)
for layer in pretrained_model.layers:
    layer.trainable = False

resnet_model.add(pretrained_model)
resnet_model.add(tf.keras.layers.Flatten())
resnet_model.add(tf.keras.layers.Dense(512, activation='relu'))
resnet_model.add(tf.keras.layers.Dropout(rate=0.3))
resnet_model.add(tf.keras.layers.Dense(7, activation='softmax'))

: 

In [ ]:
resnet_model.summary()

: 

In [ ]:
#Compile the deep neural network using the following code
learning_rate = 0.1
epochs=100
resnet_model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=[
            tf.keras.metrics.Accuracy(),
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
        ],
)

: 

In [ ]:
model_fit = resnet_model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
        ],)      

: 

In [ ]:
for _ in range(5):
    img, label = train_generator.next()
    print(img.shape)
    plt.imshow(img[0])
    plt.show()

: 

In [ ]:
from tensorflow.keras.models import Model

conv_output = neural_net.get_layer('resnet50').get_layer("conv5_block3_out").output
pred_output = neural_net.get_layer("dense").output
input_layer = neural_net.get_layer('resnet50').get_layer('input_1').input


explained_model = tf.keras.Model(input_layer, outputs=[conv_output, pred_output])

: 

In [ ]:
pred_output = neural_net.get_layer("dense_9").output

: 

In [ ]:
neural_net.get_layer('resnet50').get_layer("input_9").input

: 